In [1]:
import pandas as pd
import re
import requests, bs4
from requests import get
from bs4 import BeautifulSoup
from IPython.core.display import clear_output
from warnings import warn
from time import sleep
from random import randint
from time import time

In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}

In [3]:
step1=0
step2=50

In [4]:
pages = [str(i) for i in range(step1,step2,25)]

In [5]:
pages

['0', '25']

In [6]:
review_titles = []
review_dates = []
user_scores=[]
names = []
positions = []
companys=[]
IndustryTypes=[]
Employees=[]
review_bodys = []
review_pros = []
review_cons = []

Getting Links of all Cisco Products

In [7]:
cisco_url = get("https://www.trustradius.com/vendors/cisco", headers = headers)

In [8]:
cisco_soup_url = BeautifulSoup(cisco_url.text,'html.parser')

In [9]:
main_table = cisco_soup_url.find_all('a',class_='ProductGridProduct_product__2iNPt')

In [10]:
urls = [] 
for i in main_table:
    url = i['href']
    if not url.startswith('http'):
        url = "https://www.trustradius.com"+url
    urls.append(url)

In [11]:
urls

['https://www.trustradius.com/products/cisco-webex-meetings/reviews',
 'https://www.trustradius.com/products/cisco-catalyst/reviews',
 'https://www.trustradius.com/products/cisco-jabber/reviews',
 'https://www.trustradius.com/products/cisco-routers/reviews',
 'https://www.trustradius.com/products/cisco-asa/reviews',
 'https://www.trustradius.com/products/cisco-webex-teams/reviews',
 'https://www.trustradius.com/products/cisco-aironet-and-catalyst-access-points/reviews',
 'https://www.trustradius.com/products/cisco-unified-communications-manager/reviews',
 'https://www.trustradius.com/products/cisco-dna-center/reviews',
 'https://www.trustradius.com/products/appdynamics/reviews',
 'https://www.trustradius.com/products/cisco-sd-wan/reviews',
 'https://www.trustradius.com/products/cisco-meraki-mx/reviews',
 'https://www.trustradius.com/products/duo-security/reviews',
 'https://www.trustradius.com/products/cisco-anyconnect/reviews',
 'https://www.trustradius.com/products/cisco-4000-series-

Creating function for getting data

In [12]:
def cisco_data(url):
    requests = 0
    start_time = time()
    for page in pages:
        #make a get request
        #cisco = get('https://www.trustradius.com/products/cisco-webex-meetings/reviews?f='+page, headers = headers)
        cisco = get(url+'?f='+page,headers = headers)
        
        
        #pause the loop for 8-20 seconds
        sleep(randint(8,20))
    
        #monitor the requests
        requests += 1
        elapsed_time = time() - start_time
        print('Request:{}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
        clear_output(wait = True)
    
        #show a warning if a non 200 status code is returned
        if cisco.status_code != 200:
            warn('Request: {}; Status code: {}'.format(requests, response.status_code))
        
        
        cisco_soup = BeautifulSoup(cisco.text,'html.parser')
    
        #find the major tag peculiar to each review
        container = cisco_soup.find_all("div", class_ = 'serp-header')
        container2 = cisco_soup.find_all("div", class_ = 'serp-body')
        #iterate through the major tag 
        for con in container:
            
            #scrape the review body
            if not con.find('h3'):
                review_title='-'
            else:
                review_title = con.find('h3').text
            review_titles.append(review_title)
            
            
            #scrape the review dates
            if not con.select('div.review-date'):
                review_date='-'
            else:
                review_date = con.select('div.review-date')[0].text
            review_dates.append(review_date)
    
    
            #scrape the user scores
            if not con.select('div.trust-score__score span'):
                user_score='-'
            else:
                user_score= con.select('div.trust-score__score span')[1].text
            user_scores.append(user_score)
    
            #scrape the user names.
            if not con.select('div.name'):
                name='-'
            else:
                name = con.select('div.name')[0].text
            names.append(name)
        
            #scrape the user positions.
            if not con.select('div.position'):
                position='-'
            else:
                position = con.select('div.position')[0].text
            positions.append(position)
            
            #scrape the company
            if not con.select('span.company'):
                company='-'
            else:
                company=con.select('span.company')[0].text
            companys.append(company)

        
            #scrape the industry tupe.
            if not con.select('span.industry-type'):
                IndustryType='-'
            else:
                IndustryType = con.select('span.industry-type')[0].text
            IndustryTypes.append(IndustryType)
        
            #scrape the employees count
            if not con.select('span.size'):
                Employee='-'
            else:
                Employee = con.select('span.size')[0].text
            Employees.append(Employee)
        
            #scrape the review body
        for con in container2:
            if not con.select('div.question-response-container')[0]:
                review_body = '-'
            else:
                review_body= con.select('div.question-response-container')[0].text
            review_bodys.append(review_body)
        
        
            #scrape the review pros
            if not con.select('ul.pros'):
                review_pro='-'
            else:
                review_pro = con.select('ul.pros')[0].text
            review_pros.append(review_pro)
        
        
            #scrape the review cons
            if not con.select('ul.cons'):
                review_con='-'
            else:
                review_con = con.select('ul.cons')[0].text
            review_cons.append(review_con)
   # return review_titles,review_bodys,review_pros,review_cons,review_dates,user_scores,names,positions,IndustryTypes,Employees
        
        
        

In [13]:
urls = ['https://www.trustradius.com/products/cisco-webex-meetings/reviews',
 'https://www.trustradius.com/products/cisco-catalyst/reviews']


In [14]:
#Links pass to function
for url in urls:
    cisco_data(url)

Request:2; Frequency: 0.056980656279105965 requests/s


In [15]:
cisco_df = pd.DataFrame({'Review_Title': review_titles,
                         'Review_body':review_bodys,
                         'Review_pros':review_pros,
                         'Review_cons':review_cons,
'Review_date': review_dates,
'User_Rating': user_scores,
'Customer_Name': names,
'Position':positions,
'Company':companys,
'IndustryType':IndustryTypes,
'Number 0f Employees':Employees})
print(cisco_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68 entries, 0 to 67
Data columns (total 11 columns):
Review_Title           68 non-null object
Review_body            68 non-null object
Review_pros            68 non-null object
Review_cons            68 non-null object
Review_date            68 non-null object
User_Rating            68 non-null object
Customer_Name          68 non-null object
Position               68 non-null object
Company                68 non-null object
IndustryType           68 non-null object
Number 0f Employees    68 non-null object
dtypes: object(11)
memory usage: 6.0+ KB
None


In [16]:
cisco_df

,Review_Title,Review_body,Review_pros,Review_cons,Review_date,User_Rating,Customer_Name,Position,Company,IndustryType,Number 0f Employees
0,Cisco Webex Meetings,I'm using Cisco Webex Meetings almost daily fo...,"During troubleshooting sessions, I'm able to t...",Sound quality during meetings when I'm in a lo...,"September 04, 2020",10 out of 10,Joacim Wicander,Senior Data Center Engineer,Conscia Sverige AB,Information Technology & Services,51-200 employees
1,Cisco Webex Meetings Review,Our organization relies on Cisco Webex Meeting...,Easy to useHas better featuresRecording capabi...,I don't see any cons at this time.,"August 31, 2020",10 out of 10,Zar Jerome Cajudo,System Engineer,"Trends & Technologies, Inc.",Retail,501-1000 employees
2,Cisco Webex Meetings for Meeting and Training ...,Cisco Webex Meetings is used for our day to da...,SimplicityMany featuresEasy to useIntegration ...,Sharing file using Webex MeetingsBandwidth opt...,"August 24, 2020",9 out of 10,Budi Ezeddin,Senior Technical Specialist,Synnex Metrodata Indonesia,Information Technology & Services,201-500 employees
3,Webex Meetings will outshine the rest.....hand...,We use it across the entire org and it support...,One button to push joining. Just having one bu...,The back end management of users CAN be a bit ...,"April 07, 2020",10 out of 10,Joshua Baever,Sr. Systems Administrator,T-Mobile,Telecommunications,"10,001+ employees"
4,Webex is #1 in my book!,I started using the personal one when my compa...,Easy implementationWebex well known throughout...,After permission is given for keyboard/mouse c...,"May 28, 2020",9 out of 10,Sandt Brian,System/DB/UCCE Admin,HBCS,Hospital & Health Care,501-1000 employees
...,...,...,...,...,...,...,...,...,...,...,...
63,Catalyst 2960 fond adieu,Catalyst 2960-L SeriesCatalyst 3560-CX Series,Connect up students to the routersCope with ge...,These switches are EOL but they are still perf...,"March 05, 2019",5 out of 10,Peter Mckenzie,Cisco Instructor,Chisholm Institute,Education Management,1001-5000 employees
64,Cisco 2960X switch - reliable and powerful ent...,Catalyst 2960 X/XR Series,"Wired 802.1x just works, which increases secur...",Boot time of a stack could be improvedStacking...,"March 07, 2019",10 out of 10,Darren Hosking,Solutions Architect,NEC Australia,Computer Networking,1001-5000 employees
65,The Pros and Bugs of Cisco Switches,Catalyst 3850 Series,Cisco 3850s are well supported throughout the ...,Cisco 3850s code upgrades have introduced many...,"March 07, 2019",7 out of 10,Verified User,Technician in Information Technology,-,Hospital & Health Care Company,1001-5000 employees
66,Great Product,Catalyst 2960 X/XR SeriesCatalyst 3650 SeriesC...,Campus core switchingCampus access switchingCa...,Rolling software updates with no downtimeMore ...,"March 07, 2019",9 out of 10,Verified User,Engineer in Information Technology,-,Professional Training & Coaching Company,1001-5000 employees
